In [77]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col , lit , broadcast , count

# Build initial spark session

In [78]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("HospitalData")
    .config("spark.sql.adaptive.enabled", "false")
    .config("spark.sql.autoBroadcastJoinThreshold", -1)
    .config("spark.dynamicAllocation.enabled", "true")
    .config("spark.shuffle.service.enabled", "true")
    .config("spark.dynamicAllocation.minExecutors", 2)
    .config("spark.dynamicAllocation.initialExecutors", 4)
    .config("spark.dynamicAllocation.maxExecutors", 10)
    .getOrCreate()
)


In [79]:
spark

# Read data

In [80]:
patients = spark.read.format("csv") \
    .option("header" , "true") \
    .option("inferSchema" , "true") \
    .load("/home/jovyan/csv_datas/patients.csv")

In [81]:
billings = spark.read.format("csv") \
    .option("header" , "true") \
    .option("inferSchema" , "true") \
    .load("/home/jovyan/csv_datas/billing.csv")

In [82]:
appointments = spark.read.format("csv") \
    .option("header" , "true") \
    .option("inferSchema" , "true") \
    .load("/home/jovyan/csv_datas/appointments.csv")

In [83]:
doctors = spark.read.format("csv") \
    .option("header" , "true") \
    .option("inferSchema" , "true") \
    .load("/home/jovyan/csv_datas/doctors.csv")

In [84]:
treatments = spark.read.format("csv") \
    .option("header" , "true") \
    .option("inferSchema" , "true") \
    .load("/home/jovyan/csv_datas/treatmentss.csv")

# Partioning files

In [85]:
patients = patients.repartition(10)
billings = billings.repartition(10)
treatments = treatments.repartition(10)
appointments = appointments.repartition(10)

In [86]:
patients.rdd.getNumPartitions()

10

In [87]:
appointments.rdd.getNumPartitions()

10

In [88]:
treatments.rdd.getNumPartitions()

10

In [89]:
billings.rdd.getNumPartitions()

10

# Save them as parquet and also by partitioning

In [90]:
# patients.write.format('parquet') \
#     .mode("overwrite") \
#     .option("path" , "/home/jovyan/partitioned-data/patients") \
#     .partitionBy("insurance_provider") \
#     .save()

In [91]:
# billings.write.format('parquet') \
#     .mode("overwrite") \
#     .option("path" , "/home/jovyan/partitioned-data/billings") \
#     .partitionBy("payment_status") \
#     .save()

In [92]:
# treatments.write.format('parquet') \
#     .mode("overwrite") \
#     .option("path" , "/home/jovyan/partitioned-data/treatments") \
#     .partitionBy("treatment_type") \
#     .save()

In [93]:
# appointments.write.format('parquet') \
#     .mode("overwrite") \
#     .option("path" , "/home/jovyan/partitioned-data/appointments") \
#     .partitionBy("reason_for_visit") \
#     .save()

# Read partitioneddata

In [99]:
patients = spark.read.format("parquet") \
    .load("/home/jovyan/partitioned-data/patients")

In [100]:
appointments = spark.read.format("parquet") \
    .load("/home/jovyan/partitioned-data/appointments")

In [101]:
treatments = spark.read.format("parquet") \
    .load("/home/jovyan/partitioned-data/treatments")

In [102]:
billings = spark.read.format("parquet") \
    .load("/home/jovyan/partitioned-data/billings")

# Join doctors to appointments by broadcast (Cause doctors is too small)

In [104]:
appointments_doctros_join = appointments.join(broadcast(doctors) , appointments["doctor_id"]==doctors["doctor_id"] , 'left')

In [107]:
patients.groupBy("patient_id").agg(count()).show()

NameError: name 'count' is not defined